In [26]:
# Importing Dependencies
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

#Import API Key
from api_key import wx_api_key

In [27]:
# Creating List of Dates
dates = []

# Assuming 365 Days in a Year
number_of_days = 365

# Inputting Start Date
start_date_obj = datetime(2015, 1, 1)

# For Loop to Create List of Dates
for day in range(number_of_days):
    new_date = start_date_obj + timedelta(day)
    new_date = new_date.strftime("%Y-%m-%d")
    dates.append(new_date)

In [28]:
# Reading in CSV File
filepath = '../Resources/lg_arprts.csv'
f = pd.read_csv(filepath)
df = pd.DataFrame(f)
df = df.drop(columns=['Unnamed: 0']) # Dropping Column
iata_codes = df['iata_code']

# List of Major Airports in New York
locs = []

# Looping Through Iata Codes
for code in iata_codes:
    locs.append(code)

# The API Calls Out with 5 IATAs, Limiting to Two
locs = [locs[1], locs[2]]


###  Perform API Calls

In [29]:
# Base URL
base_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"

# WorldWeather Online Has Two Formats with XML as the Default
format_output = 'json'

# Time Interval, Set to 6 Hourly Intervals
tp = 6

# Include location
incLoc = 'yes'

# Building Query URL
query_url = f"{base_url}key={wx_api_key}&format={format_output}&tp={tp}&includelocation={incLoc}"

In [30]:
# Creating Empty Lists
date_list = []
time_list = []
loc_list = []
lat =[]
lng = []
tempF = []
DewPointF = []
windspeedMiles = []
WindGustMiles = []
winddir16Point = []
weatherDesc = []
precipInches = []
humidity = []
visibilityMiles = []
pressure = []
cloudcover = []
HeatIndexF = []
WindChillF = []
totalSnow_cm = []

# Using a For Loop to Make Successive API Calls
print('Beginning Data Retrieval')
print('------------------------')

# Loop Through Each Time Interval (*Hard Coded for 6 Hourly Data*)
for time in range(4):
    # Loop Through Dates        
    for date in dates:
        # Loop Through IATA Codes
        for loc in locs:
            # Making Requests
            response = requests.get(f'{query_url}&date={date}&q={loc}').json()
            
            # Using Try/Except to Grab Variables if Available
            try:
                print(f'Processing Time Interval {time} for {date} for {loc}')
                loc_list.append(loc)
                date_list.append(date)
                time_list.append(response['data']['weather'][0]['hourly'][time]['time'])
                lat.append(response['data']['nearest_area'][0]['latitude'])
                lng.append(response['data']['nearest_area'][0]['longitude'])
                totalSnow_cm.append(response['data']['weather'][0]['totalSnow_cm'])
                tempF.append(response['data']['weather'][0]['hourly'][time]['tempF'])
                DewPointF.append(response['data']['weather'][0]['hourly'][time]['DewPointF'])
                windspeedMiles.append(response['data']['weather'][0]['hourly'][time]['windspeedMiles'])
                WindGustMiles.append(response['data']['weather'][0]['hourly'][time]['WindGustMiles'])
                winddir16Point.append(response['data']['weather'][0]['hourly'][time]['winddir16Point'])
                weatherDesc.append(response['data']['weather'][0]['hourly'][time]['weatherDesc'][0]['value'])
                precipInches.append(response['data']['weather'][0]['hourly'][time]['precipInches'])
                humidity.append(response['data']['weather'][0]['hourly'][time]['humidity'])
                visibilityMiles.append(response['data']['weather'][0]['hourly'][time]['visibilityMiles'])
                pressure.append(response['data']['weather'][0]['hourly'][time]['pressure'])
                cloudcover.append(response['data']['weather'][0]['hourly'][time]['cloudcover'])
                HeatIndexF.append(response['data']['weather'][0]['hourly'][time]['HeatIndexF'])
                WindChillF.append(response['data']['weather'][0]['hourly'][time]['WindChillF'])
            except KeyError:
                print('Day not found. Skipping...')
print('------------------------')
print('Data Retrieval Complete')
print('------------------------')

Beginning Data Retrieval
------------------------
Processing Time Interval 0 for 2015-01-01 for JFK
Processing Time Interval 0 for 2015-01-01 for LGA
Processing Time Interval 0 for 2015-01-02 for JFK
Processing Time Interval 0 for 2015-01-02 for LGA
Processing Time Interval 0 for 2015-01-03 for JFK
Processing Time Interval 0 for 2015-01-03 for LGA
Processing Time Interval 0 for 2015-01-04 for JFK
Processing Time Interval 0 for 2015-01-04 for LGA
Processing Time Interval 0 for 2015-01-05 for JFK
Processing Time Interval 0 for 2015-01-05 for LGA
Processing Time Interval 0 for 2015-01-06 for JFK
Processing Time Interval 0 for 2015-01-06 for LGA
Processing Time Interval 0 for 2015-01-07 for JFK
Processing Time Interval 0 for 2015-01-07 for LGA
Processing Time Interval 0 for 2015-01-08 for JFK
Processing Time Interval 0 for 2015-01-08 for LGA
Processing Time Interval 0 for 2015-01-09 for JFK
Processing Time Interval 0 for 2015-01-09 for LGA
Processing Time Interval 0 for 2015-01-10 for JFK


ConnectionError: HTTPSConnectionPool(host='api.worldweatheronline.com', port=443): Max retries exceeded with url: /premium/v1/past-weather.ashx?key=6f4c63d9ec5f4745a53172620213001&format=json&tp=6&includelocation=yes&date=2015-02-13&q=LGA (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000023E04780F10>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

### Create DataFrame

In [32]:
 # Creating DataFrame
historical_wx_df = pd.DataFrame({
    'Airport Code': loc_list,
    'Date': date_list,
    'Time': time_list,
    'Latitude': lat,
    'Longitude': lng,
    'Weather Description': weatherDesc,
    'Temperature (F)': tempF,
    'Heat Index (F)': HeatIndexF,
    'Wind Chill (F)': WindChillF,
    'Dew Point (F)': DewPointF,
    'Wind Speed (mph)': windspeedMiles,
    'Wind Gusts (mph)': WindGustMiles,
    'Wind Direction': winddir16Point,
    'Precipitation (in)': precipInches,
    'Humidity (%)': humidity,
    'Visibility (mi)': visibilityMiles,
    'Pressure (mb)': pressure,
    'Cloud Coverage (%)': cloudcover,
    'Snow Total (cm)': totalSnow_cm
})

historical_wx_df

,Airport Code,Date,Time,Latitude,Longitude,Weather Description,Temperature (F),Heat Index (F),Wind Chill (F),Dew Point (F),Wind Speed (mph),Wind Gusts (mph),Wind Direction,Precipitation (in),Humidity (%),Visibility (mi),Pressure (mb),Cloud Coverage (%),Snow Total (cm)
0,JFK,2015-01-01,0,40.640,-73.779,Clear,30,31,20,17,14,19,WSW,0.0,57,6,1022,19,0.0
1,LGA,2015-01-01,0,40.776,-73.873,Clear,28,28,19,15,11,16,WSW,0.0,58,6,1021,13,0.0
2,JFK,2015-01-02,0,40.640,-73.779,Clear,37,37,28,25,15,24,WSW,0.0,62,6,1017,6,0.0
3,LGA,2015-01-02,0,40.776,-73.873,Clear,35,35,26,23,12,21,WSW,0.0,63,6,1017,6,0.0
4,JFK,2015-01-03,0,40.640,-73.780,Overcast,35,36,29,26,9,14,NNE,0.0,69,6,1032,100,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,JFK,2015-02-11,0,40.640,-73.780,Partly cloudy,28,28,17,25,15,23,NNE,0.0,86,6,1017,36,0.0
83,LGA,2015-02-11,0,40.780,-73.870,Clear,26,26,14,23,13,21,N,0.0,88,6,1018,13,0.0
84,JFK,2015-02-12,0,40.640,-73.780,Partly cloudy,28,29,24,24,5,7,SSW,0.0,80,6,1013,29,0.3
85,LGA,2015-02-12,0,40.780,-73.870,Partly cloudy,25,26,20,22,5,9,SSW,0.0,86,6,1012,43,0.0


### Creating CSV File

In [6]:
# Creating CSV File
historical_wx_df.to_csv('../Resources/historical_wx.csv')

### Reading CSV File

In [7]:
 # Reading in CSV File
filepath = '../Resources/historical_wx.csv'
f = pd.read_csv(filepath)
df = pd.DataFrame(f)
df = df.drop(columns=['Unnamed: 0']) # Dropping Column

df.head()

,Date,Time,Latitude,Longitude,Weather Code,Weather Description,Temperature (F),Heat Index (F),Wind Chill (F),Dew Point (F),Wind Speed (mph),Wind Gusts (mph),Wind Direction,Precipitation (in),Humidity (%),Visibility (mi),Pressure (mb),Cloud Coverage (%),Snow Total (cm)
0,2015-01-01,0,40.714,-74.006,113,Clear,27,28,19,15,10,14,WSW,0.0,59,6,1022,10,0.0
1,2015-01-02,0,40.714,-74.006,113,Clear,35,36,27,24,13,21,WSW,0.0,63,6,1018,6,0.0
2,2015-01-03,0,40.714,-74.006,122,Overcast,32,33,26,25,8,14,NNE,0.0,74,6,1032,100,0.0
3,2015-01-04,0,40.714,-74.006,143,Mist,43,44,42,43,3,5,N,0.0,97,1,1018,100,0.0
4,2015-01-05,0,40.714,-74.006,113,Clear,43,43,36,38,13,22,W,0.0,83,6,1011,3,0.0
